<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/self_recursive_agi_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
self_recursive_agi.py

A self-contained PyTorch script demonstrating:
- Synthetic regression dataset
- Feed-forward AGI model with outer-loop training and inner-loop self-improvement
- Gradient clipping, LR scheduling, checkpointing, and GPU support
- Progress logging with tqdm
"""

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm


# -----------------------------------------------------------------------------
# 1. Synthetic Regression Dataset
# -----------------------------------------------------------------------------
class SyntheticRegressionDataset(Dataset):
    """
    Generates random (X, y) pairs for a linear regression task
    with additive Gaussian noise.
    """
    def __init__(self, num_samples=2000, input_size=10):
        super().__init__()
        self.X = torch.randn(num_samples, input_size)
        true_w = torch.randn(input_size, 1)
        self.y = self.X @ true_w + 0.1 * torch.randn(num_samples, 1)

    def __len__(self):
        return self.X.size(0)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


def get_dataloader(batch_size=32, input_size=10, num_samples=2000):
    dataset = SyntheticRegressionDataset(num_samples, input_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


# -----------------------------------------------------------------------------
# 2. Model Definition
# -----------------------------------------------------------------------------
class SelfRecursiveAGI(nn.Module):
    """
    Simple feed-forward net with:
      - outer-loop optimizer for standard training
      - inner-loop optimizer for self-improvement on the same batch
    """
    def __init__(self,
                 input_size: int,
                 hidden_size: int,
                 output_size: int,
                 lr_main: float = 1e-3,
                 lr_self: float = 1e-3):
        super().__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)

        # Outer‐loop optimizer
        self.main_optimizer = optim.Adam(self.parameters(), lr=lr_main)
        # Inner‐loop (self‐improve) optimizer
        self.self_optimizer = optim.Adam(self.parameters(), lr=lr_self)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h = torch.relu(self.hidden(x))
        return self.output(h)

    def self_improvement(self,
                         loss_fn,
                         x: torch.Tensor,
                         y: torch.Tensor,
                         steps: int = 5,
                         clip_grad_norm: float = 1.0) -> float:
        """
        Performs `steps` mini‐updates on the same batch to
        simulate recursive self-improvement.
        Returns the last self-improvement loss.
        """
        last_loss = 0.0
        for _ in range(steps):
            self.self_optimizer.zero_grad()
            preds = self.forward(x)
            loss = loss_fn(preds, y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.parameters(), clip_grad_norm)
            self.self_optimizer.step()
            last_loss = loss.item()
        return last_loss


# -----------------------------------------------------------------------------
# 3. Training + Self-Improvement Loop
# -----------------------------------------------------------------------------
def train_and_self_improve(model: SelfRecursiveAGI,
                           dataloader: DataLoader,
                           loss_fn,
                           device: torch.device = torch.device('cpu'),
                           epochs: int = 20,
                           self_steps: int = 3,
                           clip_grad_norm: float = 1.0,
                           scheduler_step: int = 5,
                           scheduler_gamma: float = 0.5,
                           checkpoint_dir: str = 'checkpoints',
                           checkpoint_interval: int = 5):
    os.makedirs(checkpoint_dir, exist_ok=True)
    model.to(device)
    scheduler = StepLR(model.main_optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0

        progress = tqdm(dataloader, desc=f"[Epoch {epoch}/{epochs}]")
        for X_batch, y_batch in progress:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            # Outer-loop update
            model.main_optimizer.zero_grad()
            preds = model(X_batch)
            loss_main = loss_fn(preds, y_batch)
            loss_main.backward()
            nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
            model.main_optimizer.step()

            # Inner-loop self-improvement
            loss_self = model.self_improvement(
                loss_fn,
                X_batch,
                y_batch,
                steps=self_steps,
                clip_grad_norm=clip_grad_norm
            )

            epoch_loss += loss_main.item()
            progress.set_postfix({
                'loss_main': f"{loss_main.item():.4f}",
                'loss_self': f"{loss_self:.4f}"
            })

        scheduler.step()
        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch} completed. Avg outer-loop loss: {avg_loss:.6f}")

        # Checkpointing
        if epoch % checkpoint_interval == 0:
            ckpt_path = os.path.join(checkpoint_dir, f"agi_epoch{epoch}.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': model.main_optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict()
            }, ckpt_path)
            print(f"Checkpoint saved: {ckpt_path}")


# -----------------------------------------------------------------------------
# 4. Entry Point
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Hyperparameters
    BATCH_SIZE = 32
    INPUT_SIZE = 10
    HIDDEN_SIZE = 64
    OUTPUT_SIZE = 1
    LR_MAIN = 1e-3
    LR_SELF = 1e-3
    EPOCHS = 20
    SELF_STEPS = 3
    CLIP_GRAD_NORM = 1.0
    SCHED_STEP = 5
    SCHED_GAMMA = 0.5
    CHECKPOINT_DIR = 'checkpoints'
    CHECKPOINT_INTERVAL = 5

    # Setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataloader = get_dataloader(batch_size=BATCH_SIZE, input_size=INPUT_SIZE)

    model = SelfRecursiveAGI(
        input_size=INPUT_SIZE,
        hidden_size=HIDDEN_SIZE,
        output_size=OUTPUT_SIZE,
        lr_main=LR_MAIN,
        lr_self=LR_SELF
    )
    loss_fn = nn.MSELoss()

    # Train
    train_and_self_improve(
        model=model,
        dataloader=dataloader,
        loss_fn=loss_fn,
        device=device,
        epochs=EPOCHS,
        self_steps=SELF_STEPS,
        clip_grad_norm=CLIP_GRAD_NORM,
        scheduler_step=SCHED_STEP,
        scheduler_gamma=SCHED_GAMMA,
        checkpoint_dir=CHECKPOINT_DIR,
        checkpoint_interval=CHECKPOINT_INTERVAL
    )